In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as pyo
from plotly.graph_objs import * 
import plotly.plotly as py
pyo.offline.init_notebook_mode()

read pickle

In [ ]:
dataset1 = pd.read_pickle('F_pickle') 
dataset2 = pd.read_pickle('M_pickle')

In [ ]:
data1 = pd.DataFrame(dataset1)
data2 = pd.DataFrame(dataset2)


rename variables.

In [ ]:
data1.columns = ['F_M1','F_M2','F_P1','F_p2']
data2.columns = ['M_M1','M_M2']

merge by index.

In [ ]:
data = pd.merge(data1,data2,left_index=True,right_index=True) 
data_pure = data.dropna(axis = 0 , how = 'any' )

In [ ]:
subdata = pd.merge(data_pure["F_p2"].to_frame(),data_pure["F_p2"].to_frame(), left_index=True, right_index=True)

In [ ]:
subdata.ix[0,0]= (subdata.ix[0,0]+1)

In [ ]:
subdata["F_p2_x"] =subdata["F_p2_x"]*100

In [ ]:
subdata["F_p2_x"] = subdata["F_p2_x"].cumsum()

In [ ]:
subdata["F_p2_y"] =(subdata["F_p2_y"]+1).cumprod()*100

In [ ]:
subdata.reset_index(inplace=True)
subdata.columns = ["time",'F_p2_single_value','F_p2_compound_value']

In [ ]:
subdata.head(3)

In [ ]:
def createStackedQuantArea(df, time, cols, hover, title, yaxisTitle):
    traces = []
    stackedAreaDF = df.loc[:, ([time] + cols)]
    stackedAreaDF.fillna(0, inplace=True)
    
    cumulative = stackedAreaDF[cols].cumsum(axis = 1)
    
    cumulAndOrig = cumulative.merge(stackedAreaDF, 
                                         left_index = True,
                                         right_index = True,
                                        suffixes = ('_c','_o'))
    
    for col in cols:
        cumulAndOrig[col + '_t'] = "<b>" + str(col)[:-6]  + "</b><br>" + str(hover) + cumulAndOrig[col + "_o"].apply(lambda x:
            "{:,}".format(int(round(x, 0))))
        
        traces.append({'type' : 'scatter',
                      'x' : cumulAndOrig[time],
                      'y' : cumulAndOrig[col + "_c"],
                       'text' : cumulAndOrig[col + "_t"],
                       'hoverinfo' : 'text+x',
                      'name' : col[:-6],
                      'mode' : 'lines',
                      'fill' : 'tonexty'})
        
    data = Data(traces)
    layout = {'title' : title,
             'xaxis' : {'title' : time},
             'yaxis' : {'title' : yaxisTitle,
                       'tickprefix' : '$'},
             'hovermode' : 'closest'}
    fig = Figure(data = data, layout = layout)
    pyo.iplot(fig)
    return cumulAndOrig,fig
    

In [ ]:
data, fig = createStackedQuantArea(subdata,"time",['F_p2_single_value','F_p2_compound_value'],'Total Fund: ',"Accumulated value of single and that of compound",'Fund Of Single and Compound')

In [ ]:
py.plot(fig, filename="Accumulated value of single and compound", fileopt="overwrite")

In [ ]:
def createStackedPropArea(df, time, cols, hover, title, yaxisTitle): 
    PCcols = []
    traces = []
    
    stackedAreaDF = df.loc[:, ([time] + cols)]
    stackedAreaDF.fillna(0, inplace=True)
        
    stackedAreaDF['Total'] = stackedAreaDF[cols].sum(axis =1)
    
    for col in cols:
        stackedAreaDF["pc_"+str(col)] = stackedAreaDF[col] / stackedAreaDF['Total']
        PCcols.append("pc_"+str(col))
        
    stackedPCAreaDF = stackedAreaDF[PCcols].cumsum(axis=1)
    stackedAreaData = stackedAreaDF[PCcols + [time]].merge(stackedPCAreaDF[PCcols], 
                                        left_index = True,
                                         right_index = True,
                                        suffixes = ('_o','_c'))

    for col in PCcols:       
        stackedAreaData[col + '_t'] = "<b>" + str(col)[3:-6]  + "</b><br>" + str(hover) + stackedAreaData[col + "_o"].apply(lambda x:
            "{:.0%}".format(x))
        
        
        traces.append({'type' : 'scatter',
                      'x' : stackedAreaData[time],
                      'y' : stackedAreaData[col + "_c"],
                       'text' : stackedAreaData[col + "_t"],
                       'hoverinfo' : 'text+x',
                      'name' : col[3:-6],
                      'mode' : 'lines',
                      'fill' : 'tonexty'})
    
    data = Data(traces)
    layout = {'title' : title,
             'xaxis' : {'title' : time},
             'yaxis' : {'title' : yaxisTitle,
                       'tickformat' : '%'},
              'hovermode' : 'closest'}
    fig = Figure(data = data, layout = layout)
    pyo.iplot(fig)

    return stackedAreaData,fig
    


In [ ]:
data, fig = createStackedPropArea(subdata,"time",['F_p2_single_value','F_p2_compound_value'],'Odds of Fund: ',"Odds of value of Single and value of Compound",'Odds of Fund Of Single and Compound')

In [ ]:
py.plot(fig, filename="Odds of value of Single and value of Compound" ,fileopt="overwrite")